### Play around with fitting polynomials to photometry 

Import stuff

In [ ]:
import os
import tables_io
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

from macss import plotting_functions
from macss import utility_functions


#### Change this to be the correct locations

In [ ]:
HOME = os.environ['HOME']
pz_dir = f'{HOME}/macss'

Get the data, and split it

In [ ]:
d = tables_io.read(f"{pz_dir}/data/roman_rubin_9925.hdf5")
train = tables_io.sliceObj(d, slice(0, -1, 5))
test = tables_io.sliceObj(d, slice(1, -1, 5))

#### Note that this simulated catalog goes far deeper than we can observe

In [ ]:
np.nanmax(d['LSST_obs_g'])

Get the colors & mags

In [ ]:
train_targets, train_features = utility_functions.prepare_data_total_mag_and_colors(train, 'LSST_obs_{band}', 'ugrizy')
test_targets, test_features = utility_functions.prepare_data_total_mag_and_colors(test, 'LSST_obs_{band}', 'ugrizy')

train_mags = utility_functions.get_band_values(train, 'LSST_obs_{band}', 'ugrizy')
train_mag_errs = utility_functions.get_band_values(train, 'LSST_obs_{band}_err', 'ugrizy')


Pick some medians for the filters, note: these are just estimates

In [ ]:
filter_medians = np.array([350., 470., 620., 750., 870., 950.])
x_piv = 0.5*(filter_medians[0] + filter_medians[-1])

Plot lots of spectra, color coded by redshift

In [ ]:
_ = plotting_functions.plot_mag_spectra(filter_medians, train_mags[::100,:], train_mag_errs[::100,:], train_targets[::100])

Make a single plot as an example

In [ ]:
from numpy.polynomial import Polynomial
p = Polynomial.fit(filter_medians-x_piv, train_mags[0], deg=3)

In [ ]:
resid = train_mags[0] - p(filter_medians-x_piv)

In [ ]:
resid

In [ ]:
grid = np.linspace(filter_medians[0], filter_medians[-1], 61) 
plt.errorbar(filter_medians, train_mags[0], train_mag_errs[0])
plt.plot(grid, p(grid-x_piv))

In [ ]:
p.coef

#### The next cells are just playing around with the data, looking for something interesting

Compute how much each color exceeds expectations

In [ ]:
color_excess = utility_functions.color_excess(train_mags)

Do linear fits and return residuals

In [ ]:
_lin_resids = utility_functions.linear_fit_residuals(filter_medians, train_mags, x_piv)

Plot residuals as a function of redshift

In [ ]:
_ = plt.hist2d(train_targets, np.nan_to_num(_lin_resids[:,1], 0.), bins=(100, 100), norm='log')

In [ ]:
_ = plt.hist2d(train_targets, np.nan_to_num(_lin_resids[:,4], 0.), bins=(100, 100), norm='log')

Resid v resid plot

In [ ]:
_ = plt.scatter(_lin_resids[:,3], _lin_resids[:,5], color=cm.rainbow(train_targets/3.), marker='.', s=1)